In [ ]:
''''In tis main file we do an initial explortation, we clean the data, and perfoirm EDA and vizualisation'''

import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import initial_exploration as explo
import data_cleaning as cl

%load_ext autoreload
%autoreload 2

pd.set_option('display.max_columns', None)

In [ ]:
# Load datasets:
cash_requests = pd.read_csv('project_dataset/extract - cash request - data analyst.csv')
fees = pd.read_csv('project_dataset/extract - fees - data analyst - .csv')

In [ ]:
cash_requests.head()

In [ ]:
# Inital exploration:
explo.check(cash_requests)

In [ ]:
cl.convert_dates(cash_requests, cl.cash_request_date_columns)

# Re-check and ensure correct data types in cash_requests:
cl.ensure_correct_data_types(cash_requests, cl.cash_request_date_columns)

cash_requests.head()

In [ ]:
explo.check(cash_requests)

In [ ]:
fees.head()

In [ ]:
explo.check(fees)

In [ ]:
cl.convert_dates(fees, cl.fees_data_date_columns)

# Re-check and ensure correct data types in cfees:
cl.ensure_correct_data_types(fees, cl.fees_data_date_columns)

fees.head()

In [ ]:
cl.rename_col(cash_requests, 'id', 'cash_request_id')
data_df = cl.merge_df(cash_requests, fees, 'outer', 'cash_request_id')
data_df = cl.rename_col_xy(data_df)
data_df.head(20)

In [ ]:
data_df.sample(10)

In [ ]:
cl.rename_col(data_df, 'id', 'id_fee')
cl.clean_text_column(data_df, "reason")
explo.check(data_df)

In [ ]:
data_df = cl.remove_nan(data_df, "cash_request_id")
explo.check_null(data_df)

In [ ]:
data_df = cl.drop_col(data_df, ["id_fee", "category"])
data_df.head(10)

In [ ]:
explo.check_null(data_df)

In [ ]:
frames = cl.selecting_data_types(data_df)
num = frames[1]
cat = frames[0]
date = frames[2]

In [ ]:
num.head()

In [ ]:
explo.check_null(num)

In [ ]:
cat.head()

In [ ]:
explo.check_null(cat)

In [ ]:
date.head()

In [ ]:
explo.check_null(date)

In [ ]:
# Verify specific date columns' data types in cash_requests:
print("\nCheck data types of date columns:")
print(cash_requests[['created_at', 'updated_at', 'moderated_at', 'reimbursement_date', 
                     'cash_request_received_date', 'money_back_date', 'send_at', 'reco_creation', 'reco_last_update']].dtypes)

# Check for any NaT in date columns:
print("\nCheck NaT (missing values) in date columns:")
print(cash_requests[['created_at', 'updated_at', 'moderated_at', 'reimbursement_date', 
                     'cash_request_received_date', 'money_back_date', 'send_at', 'reco_creation', 'reco_last_update']].isna().sum())

In [ ]:
df_time_between_CR_and_money_back = cl.process_date_columns(date, "CR_created_at", "money_back_date")
df_time_between_CR_and_money_back.head()

In [ ]:
explo.check(df_time_between_CR_and_money_back)

In [ ]:
df_time_between_fee_created_and_paid_at_date = cl.process_date_columns(date, "fee_created_at", "paid_at")
df_time_between_fee_created_and_paid_at_date.head(20)

In [ ]:
explo.check(df_time_between_fee_created_and_paid_at_date)

In [ ]:
# We can divide the columns between us and ecah work in a set of columns performing an 
# univariate analysis (frequency plots, box plots, doughnut, histograms   or any other you concider)
#For numercial plots also create report in mean , meadian, std, and arrive to conclusions 

In [ ]:
print(cat.columns)


In [ ]:
invalid_dates = cat[~cat['charge_moment'].apply(pd.to_datetime, errors='coerce').notna()]
print(invalid_dates)


In [ ]:
missing_count = cat[cat['charge_moment'].isna()].shape[0]
print(f'Number of rows with missing charge_moment: {missing_count}')


In [ ]:
# Ensure 'charge_moment' is in datetime format, handle invalid date strings by converting them to NaT
cat['charge_moment'] = pd.to_datetime(cat['charge_moment'], errors='coerce')

# Step 1: Define cohorts based on the month of the first cash request
cat['cohort_month'] = cat['charge_moment'].dt.to_period('M')

# Step 2: Clean the 'reason' column (strip whitespace and convert to lowercase)
cat['reason'] = cat['reason'].str.strip().str.lower()

# Step 3: Define payment incident reasons (in lowercase)
incident_reasons = ['rejected direct debit', 'month delay on payment']

# Step 4: Filter rows where 'reason' matches one of the incident reasons, and drop NaN values
incident_data = cat[cat['reason'].isin(incident_reasons)].dropna(subset=['reason'])

# Check if any incidents are found after filtering
if incident_data.empty:
    print("Incident data is empty. Inspecting data:")
    print(cat[['reason', 'cohort_month']].head(20))  # Check the first 20 rows for any reason matches

# Step 5: Group by cohort (cohort_month) and calculate the number of incidents for each cohort
incident_rate = incident_data.groupby('cohort_month').size().reset_index(name='incident_count')

# Step 6: Count the total number of requests for each cohort (including both completed and incomplete requests)
total_requests = cat.groupby('cohort_month').size().reset_index(name='total_requests')

# Step 7: Merge the incident count and total requests data
cohort_data = pd.merge(incident_rate, total_requests, on='cohort_month', how='left')

# Step 8: Calculate incident rate (number of incidents divided by the total number of requests in that cohort)
cohort_data['incident_rate'] = cohort_data['incident_count'] / cohort_data['total_requests']

# Step 9: Fill any missing values in incident_count with 0 (no incidents for that cohort)
cohort_data['incident_count'] = cohort_data['incident_count'].fillna(0)

# Display the results
print(cohort_data)


In [ ]:
# Check unique values in the 'reason' column
print(cat['reason'].unique())


In [ ]:
# Check for incidents by filtering directly on 'reason'
incident_data_check = cat[cat['reason'].isin(incident_reasons)]
print(incident_data_check.head())


In [ ]:
# Check for any NaT values in 'charge_moment'
print(cat[cat['charge_moment'].isna()])  # Rows where charge_moment could not be converted


In [ ]:
# Check for missing values in the 'reason' column
print(cat[cat['reason'].isna()])


In [ ]:
# Check the total number of requests per cohort
total_requests_check = cat.groupby('cohort_month').size().reset_index(name='total_requests')
print(total_requests_check)


In [ ]:
# Handle NaT values by dropping rows with missing 'charge_moment' and reattempt cohort calculation

# Drop rows where 'charge_moment' is NaT (you can also choose to fill them as needed)
cat_cleaned = cat.dropna(subset=['charge_moment'])

# Ensure 'charge_moment' is in datetime format, if needed
cat_cleaned['charge_moment'] = pd.to_datetime(cat_cleaned['charge_moment'], errors='coerce')

# Define cohorts based on the month of the first cash request
cat_cleaned['cohort_month'] = cat_cleaned['charge_moment'].dt.to_period('M')

# Calculate the number of requests per cohort
cohort_data = cat_cleaned.groupby('cohort_month').agg(
    total_requests=('charge_moment', 'size')
).reset_index()

# Calculate incident rate by dividing the count of incidents by the total number of requests for each cohort
cat_cleaned['incident'] = cat_cleaned['reason'].apply(lambda x: 1 if pd.notna(x) else 0)
cohort_data['incident_count'] = cat_cleaned.groupby('cohort_month')['incident'].sum().reset_index()['incident']

# Calculate incident rate
cohort_data['incident_rate'] = cohort_data['incident_count'] / cohort_data['total_requests']

# Display the cohort data
print(cohort_data)


In [ ]:
# Step 1: Drop rows where 'charge_moment' is NaT
cat_cleaned = cat.dropna(subset=['charge_moment'])

# Step 2: Convert 'charge_moment' to datetime format
cat_cleaned['charge_moment'] = pd.to_datetime(cat_cleaned['charge_moment'], errors='coerce')

# Step 3: Check if there are any valid 'charge_moment' values after conversion
print(f"Rows with valid 'charge_moment': {cat_cleaned['charge_moment'].notna().sum()}")

# Step 4: Check if the 'reason' column has any non-NaN values (for incident calculation)
print(f"Rows with valid 'reason': {cat_cleaned['reason'].notna().sum()}")

# Step 5: Create a new 'incident' column based on the 'reason'
cat_cleaned['incident'] = cat_cleaned['reason'].apply(lambda x: 1 if pd.notna(x) else 0)

# Step 6: Create cohorts based on the month of the first cash request
cat_cleaned['cohort_month'] = cat_cleaned['charge_moment'].dt.to_period('M')

# Step 7: Group by 'cohort_month' and calculate total requests and incident counts
cohort_data = cat_cleaned.groupby('cohort_month').agg(
    total_requests=('charge_moment', 'size'),
    incident_count=('incident', 'sum')
).reset_index()

# Step 8: Calculate incident rate (incident count / total requests)
cohort_data['incident_rate'] = cohort_data['incident_count'] / cohort_data['total_requests']

# Step 9: Check if cohort_data has rows
print(f"Cohort data rows: {cohort_data.shape[0]}")

# Display the result
print(cohort_data)


In [ ]:
# Step 1: Inspect the first few rows of the dataset
print("First few rows of the data:")
print(cat.head())

# Step 2: Check if 'charge_moment' and 'reason' columns are truly empty
print("\nCheck for NaN values in 'charge_moment' and 'reason':")
print(f"NaN in 'charge_moment': {cat['charge_moment'].isna().sum()} out of {len(cat)}")
print(f"NaN in 'reason': {cat['reason'].isna().sum()} out of {len(cat)}")

# Step 3: Check unique values in 'charge_moment' and 'reason' to see if they are populated or have issues
print("\nUnique values in 'charge_moment':")
print(cat['charge_moment'].unique())

print("\nUnique values in 'reason':")
print(cat['reason'].unique())

# Step 4: Check the data types of these columns to ensure they are as expected
print("\nData types of columns:")
print(cat[['charge_moment', 'reason']].dtypes)


In [ ]:
import pandas as pd

# Step 1: If charge_moment is NaT for all rows, create a cohort_month column based on the current date
cat['cohort_month'] = pd.to_datetime('now').strftime('%Y-%m')

# Step 2: Filter the dataframe where 'reason' is not NaN
filtered_data = cat[cat['reason'].notna()]

# Step 3: Group by 'cohort_month' and 'reason' and calculate total requests and incident count
cohort_data = filtered_data.groupby(['cohort_month', 'reason']).size().reset_index(name='total_requests')

# Step 4: Count the incidents by grouping on 'reason' (assuming each reason corresponds to an incident)
incident_data = filtered_data.groupby(['cohort_month', 'reason']).size().reset_index(name='incident_count')

# Step 5: Merge the data to get total_requests and incident_count in one dataframe
cohort_data = pd.merge(cohort_data, incident_data, on=['cohort_month', 'reason'], how='left')

# Step 6: Calculate the incident rate
cohort_data['incident_rate'] = cohort_data['incident_count'] / cohort_data['total_requests']

# Display the final cohort data
print(cohort_data)
